In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
from torch import nn
from torch import optim
from torchinfo import summary
from torch.utils.data import random_split, DataLoader
from torchvision.transforms import v2
import torchvision.transforms.v2.functional as FT
from torchvision.models import resnet18, ResNet18_Weights
from pathlib import Path
import wandb
import sys
import matplotlib.pyplot as plt
import numpy as np
import os
from quickai.trainer import Trainer
from quickai.utils import model_size, load_from_checkpoint
from quickai.callbacks import OverfitCallback, EarlyStoppingCallback
from quickai.logger import WandbLogger
from quickai.dataset import MapDataset

sys.path.append("../src")

from utils import plot_datapoint, plot_datapoints
from models import ResNet18
from module import ResNetModule
import settings as s
from dataset import ObjectDetectionDataset

In [3]:
data_path = Path("../data")
logs_path = Path("../logs")
logs_path.mkdir(exist_ok=True)

In [4]:
logger = WandbLogger(
    project_name=s.project_name,
    config={
        "model": s.model,
        "dataset": s.dataset,
        "max_epochs": s.max_epochs,
        "optimizer": s.optimizer,
        "lr_scheduler": s.lr_scheduler,
        "test_run": s.test_run,
        "transfer_learning": s.transfer_learning
    },
    logs_path=logs_path,
    offline=s.wandb_offline
)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [5]:
# cpu_count = os.cpu_count()
cpu_count = 7

dataset = ObjectDetectionDataset(data_path)

train_dataset, val_dataset = random_split(
    dataset, [s.dataset["train_split"], s.dataset["val_split"]]
)

# val_transforms = ResNet18_Weights.IMAGENET1K_V1.transforms()

val_transforms = v2.Compose([
    v2.Resize((224, 224)),
    # v2.CenterCrop(224),
    # Normalize
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
])

train_transforms = v2.Compose([
    # v2.RandomHorizontalFlip(),
    # v2.RandomVerticalFlip(p=0.2),
    # v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    # v2.RandomRotation(degrees=15),
    # v2.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    # v2.RandomGrayscale(p=0.1),
    
    val_transforms
])

train_dataset = MapDataset(train_dataset, transform=train_transforms)
val_dataset = MapDataset(val_dataset, transform=val_transforms)

train_dataloader = DataLoader(
    train_dataset, batch_size=s.dataset["batch_size"], shuffle=True, num_workers=cpu_count)
val_dataloader = DataLoader(
    val_dataset, batch_size=s.dataset["batch_size"],  num_workers=cpu_count)

len(dataset)

Loading from disk!


/workspaces/ObjectDetection/notebooks/../src/dataset.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.single_object_images = torch.load(data_path)
/home/codespace/.

1865

In [6]:
# index = torch.randint(0, len(train_dataset), size=(1,))
# image1, target1 = train_dataset[index]
# target1 = dataset.idx_to_class[target1]
# image1 = FT.to_image(image1).permute(1, 2, 0)

# inverse_transform = v2.Compose([
#     v2.Normalize(mean=[0., 0., 0.], std=[
#                     1 / 0.2023, 1 / 0.1994, 1 / 0.2010]),
#     v2.Normalize(mean=(-0.4914, -0.4822, -0.4465), std=[1., 1., 1.]),
# ])

# image2, target2 = train_dataset_transformed[index]
# target2 = dataset.idx_to_class[target2]

# plot_datapoints(image1, image2, target1, target2, inverse_transform2=inverse_transform)

In [7]:
callbacks = [
    EarlyStoppingCallback(min_val_accuracy=90.0, accuracy_diff=5.0, wait_epochs=5),
    # OverfitCallback(limit_batches=1, batch_size=10, max_epochs=500, augument_data=False)
]

In [8]:
model = ResNet18(num_classes=20)
summary(
    model,
    input_size=(s.dataset["batch_size"],
                *train_dataset[0][0].shape),
    device="cpu",
    mode="train",
    depth=1
)

Layer (type:depth-idx)                             Output Shape              Param #
ResNet18                                           [32, 20]                  --
├─Sequential: 1-1                                  [32, 512, 1, 1]           27,811,392
├─Sequential: 1-2                                  [32, 20]                  10,260
Total params: 27,821,652
Trainable params: 27,821,652
Non-trainable params: 0
Total mult-adds (Units.TERABYTES): 4.32
Input size (MB): 19.27
Forward/backward pass size (MB): 27334.28
Params size (MB): 111.29
Estimated Total Size (MB): 27464.84

In [8]:
module = ResNetModule(model)

optimizer = optim.AdamW(
    params=module.model.parameters(),
    weight_decay=s.optimizer["weight_decay"]
)

try:
    if s.lr_scheduler["name"] == "OneCycleLR":
        lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer=optimizer,
            max_lr=s.lr_scheduler["max_lr"],
            epochs=s.max_epochs,
            steps_per_epoch=len(train_dataloader),
        )

        print(s.lr_scheduler["name"])
except TypeError:
    lr_scheduler = None
    print("lr_scheduler is None!")

OneCycleLR


In [9]:
trainer = Trainer(
    module=module,
    logger=logger,
    optimizer=optimizer,
    callbacks=callbacks,
    logs_path=logs_path,
    fast_dev_run=s.fast_dev_run,
    limit_batches=s.limit_batches,
    lr_scheduler=lr_scheduler,
    save_checkpoint_type="best_val",
    num_workers=cpu_count
)

Using device: cuda!


In [10]:
try:
    trainer.fit(train_dataloader, val_dataloader)
except KeyboardInterrupt as e:
    print("Run stopped!")
finally:
    wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/.netrc


Time per epoch: 36.95 seconds
Epoch: 0, train_accuracy: 14.28, val_accuracy: 12.03, lr: 0.0004
Epoch: 1, train_accuracy: 17.74, val_accuracy: 18.82, lr: 0.0005
Epoch: 2, train_accuracy: 18.76, val_accuracy: 20.34, lr: 0.0008
Epoch: 3, train_accuracy: 16.35, val_accuracy: 12.33, lr: 0.0013
Epoch: 4, train_accuracy: 15.71, val_accuracy: 10.94, lr: 0.0020
Epoch: 5, train_accuracy: 16.33, val_accuracy: 3.99, lr: 0.0028
Epoch: 6, train_accuracy: 16.59, val_accuracy: 13.41, lr: 0.0037
Epoch: 7, train_accuracy: 16.48, val_accuracy: 14.09, lr: 0.0047
Epoch: 8, train_accuracy: 16.53, val_accuracy: 16.34, lr: 0.0057
Epoch: 9, train_accuracy: 17.62, val_accuracy: 17.91, lr: 0.0067
Epoch: 10, train_accuracy: 19.08, val_accuracy: 16.71, lr: 0.0076
Epoch: 11, train_accuracy: 20.87, val_accuracy: 16.86, lr: 0.0084
Epoch: 12, train_accuracy: 20.89, val_accuracy: 19.14, lr: 0.0091
Epoch: 13, train_accuracy: 22.13, val_accuracy: 17.58, lr: 0.0096
Epoch: 14, train_accuracy: 22.80, val_accuracy: 17.88, lr

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇█████
epoch_train_accuracy,▁▂▂▂▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇██
epoch_train_loss,█▇▇▇▇▇▇▇▇▇▆▆▅▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
epoch_val_accuracy,▁▃▄▁▁▂▂▃▃▃▃▃▃▄▄▅▅▂▅▄▅▅▅▅▄▆▅▅▆▅▇▇▇▇▇█████
epoch_val_loss,█▂▂▃▂▃▃▂▅▂▂▂▃▃▂▂▂▃▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁
lr,▁▁▁▂▂▄▆▆▇▇▇█████▇▇▇▆▆▆▆▆▅▄▄▄▃▃▃▃▃▂▂▁▁▁▁▁
step_train_accuracy,▁▂▅▃▃▁▅▄▃▄▃▄▂▆▄▄▄▇▄▅▅▄▆▄▆▇▄▇▄▄▆█▆▃▆▇▄▇▅▅
step_train_loss,▆▆▅▆▆▇▆▅█▅▅▇▆▅▅▅▄▅▄▅▄▄▄▆▄▄▄▄▃▄▄▄▁▃▃▅▃▄▃▁
step_val_accuracy,▄▄▅▄▅▄▅▅▃▄▄▅▃▁▆▃▄▅▅▃▅▄▅▅▇▅▅▅▅▃▃▇▅▅▆▃▆▇▆█
step_val_loss,▇▄▄▂▄▅▄▄▃▄▄▇█▄▃▃▄▂▄▃▃▃▁▃▃▃▂▁▂▂▄▂▃▃▃▂▂▃▂▃
training_step,▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█


[Metrics](https://api.wandb.ai/links/sampath017/iwrrziwg)